In [ ]:
pip install BeautifulSoup4

In [ ]:
pip install nltk

In [ ]:
!pip install gdown

In [ ]:
import pandas as pd
import string
import random
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score  
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, roc_auc_score
import matplotlib.pyplot as plt
import pandas as pd
import re
from bs4 import BeautifulSoup
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
import gdown
# Descargar los recursos de NLTK necesarios
nltk.download('punkt')
nltk.download('stopwords')



# Define el enlace compartible de Google Drive
google_drive_url = "https://drive.google.com/uc?id=153kIWdyo8JaMoaKHnQ90qigDMW1830Mg"

# Descarga el archivo desde Google Drive
gdown.download(google_drive_url, 'Final-Dataset.csv', quiet=False)

# Cargar el archivo CSV
data = pd.read_csv('Final-Dataset.csv')

# Función para limpiar el texto
def clean_text(text):
    # Eliminar etiquetas HTML
    text = BeautifulSoup(text, 'html.parser').get_text()
    # Eliminar caracteres especiales y números
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    # Tokenización
    tokens = word_tokenize(text)
    # Convertir a minúsculas y eliminar stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word.lower() for word in tokens if word.isalpha() and word.lower() not in stop_words]
    # Stemming (opcional)
    # stemmer = PorterStemmer()
    # tokens = [stemmer.stem(word) for word in tokens]
    # Unir tokens nuevamente
    cleaned_text = ' '.join(tokens)
    return cleaned_text

# Aplicar la función de limpieza a la columna de texto
data['text_cleaned'] = data['FORMATTED_CONTENT'].apply(clean_text)

# Mostrar los datos preprocesados
print(data['text_cleaned'].head())


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Crear un objeto TF-IDF Vectorizer con un número máximo de características
max_features = 5000  
tfidf_vectorizer = TfidfVectorizer(max_features=max_features)

# Ajustar y transformar los datos de texto limpio
tfidf_matrix = tfidf_vectorizer.fit_transform(data['text_cleaned'])

# Obtener las características TF-IDF en un DataFrame de pandas
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

# Imprimir las primeras filas del DataFrame para ver las características TF-IDF
print(tfidf_df.head())


In [ ]:
from sklearn.model_selection import train_test_split

# Dividir los datos en características (X) y etiquetas (y)
X = tfidf_df  # Características TF-IDF
y = data['CLASS']  # Etiquetas (clase spam o no spam)

# Dividir los datos en conjuntos de entrenamiento y pruebas (80% entrenamiento, 20% prueba)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Imprimir las dimensiones de los conjuntos de entrenamiento y pruebas
print("Dimensiones del conjunto de entrenamiento X:", X_train.shape)
print("Dimensiones del conjunto de prueba X:", X_test.shape)
print("Dimensiones del conjunto de entrenamiento y:", y_train.shape)
print("Dimensiones del conjunto de prueba y:", y_test.shape)


In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

# Crear un clasificador Naive Bayes
naive_bayes_classifier = MultinomialNB()

# Entrenar el modelo en el conjunto de entrenamiento
naive_bayes_classifier.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = naive_bayes_classifier.predict(X_test)

# Calcular la precisión del modelo
accuracy = accuracy_score(y_test, y_pred)
print("Precisión del modelo Naive Bayes:", accuracy)

# Mostrar un informe de clasificación
print(classification_report(y_test, y_pred))


In [ ]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Realizar predicciones en el conjunto de prueba para Naive Bayes
y_pred_naive_bayes = naive_bayes_classifier.predict(X_test)

# Calcular la matriz de confusión para Naive Bayes
cm_naive_bayes = confusion_matrix(y_test, y_pred_naive_bayes)

# Visualización de la matriz de confusión para Naive Bayes
plt.figure(figsize=(6, 4))
sns.heatmap(cm_naive_bayes, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.xlabel('Predicción')
plt.ylabel('Actual')
plt.title('Matriz de Confusión - Naive Bayes')
plt.show()


In [ ]:
from sklearn.linear_model import LogisticRegression

# Crear un clasificador de Regresión Logística
logistic_regression_classifier = LogisticRegression()

# Entrenar el modelo en el conjunto de entrenamiento
logistic_regression_classifier.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = logistic_regression_classifier.predict(X_test)

# Calcular la precisión del modelo
accuracy = accuracy_score(y_test, y_pred)
print("Precisión del modelo de Regresión Logística:", accuracy)

# Mostrar un informe de clasificación
print(classification_report(y_test, y_pred))


In [ ]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Realizar predicciones en el conjunto de prueba para Regresión Logística
y_pred_logistic_regression = logistic_regression_classifier.predict(X_test)

# Calcular la matriz de confusión para Regresión Logística
cm_logistic_regression = confusion_matrix(y_test, y_pred_logistic_regression)

# Visualización de la matriz de confusión para Regresión Logística
plt.figure(figsize=(6, 4))
sns.heatmap(cm_logistic_regression, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.xlabel('Predicción')
plt.ylabel('Actual')
plt.title('Matriz de Confusión - Regresión Logística')
plt.show()


In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
# Crear un clasificador Naive Bayes
naive_bayes_classifier = MultinomialNB()

# Definir los valores de alpha que deseas probar
param_grid = {'alpha': [0.1, 0.5, 1.0, 2.0]}

# Crear un objeto GridSearchCV
grid_search = GridSearchCV(naive_bayes_classifier, param_grid, cv=5, scoring='accuracy')

# Realizar la búsqueda en cuadrícula en el conjunto de entrenamiento
grid_search.fit(X_train, y_train)

# Obtener el mejor valor de alpha encontrado por la búsqueda en cuadrícula
best_alpha = grid_search.best_params_['alpha']

# Entrenar el modelo de Naive Bayes con el mejor valor de alpha
best_naive_bayes_classifier = MultinomialNB(alpha=best_alpha)
best_naive_bayes_classifier.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba con el modelo ajustado
y_pred = best_naive_bayes_classifier.predict(X_test)

# Calcular la precisión del modelo ajustado
accuracy = accuracy_score(y_test, y_pred)
print("Mejor valor de alpha:", best_alpha)
print("Precisión del modelo Naive Bayes ajustado:", accuracy)


In [ ]:
from sklearn.model_selection import GridSearchCV

# Crear un clasificador de Regresión Logística
logistic_regression_classifier = LogisticRegression()

# Definir los valores de C que deseas probar
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10]}

# Crear un objeto GridSearchCV
grid_search = GridSearchCV(logistic_regression_classifier, param_grid, cv=5, scoring='accuracy')

# Realizar la búsqueda en cuadrícula en el conjunto de entrenamiento
grid_search.fit(X_train, y_train)

# Obtener el mejor valor de C encontrado por la búsqueda en cuadrícula
best_C = grid_search.best_params_['C']

# Entrenar el modelo de Regresión Logística con el mejor valor de C
best_logistic_regression_classifier = LogisticRegression(C=best_C)
best_logistic_regression_classifier.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba con el modelo ajustado
y_pred = best_logistic_regression_classifier.predict(X_test)

# Calcular la precisión del modelo ajustado
accuracy = accuracy_score(y_test, y_pred)
print("Mejor valor de C:", best_C)
print("Precisión del modelo de Regresión Logística ajustado:", accuracy)
